You may need to install some libraries. Uncomment / edit relevant stuff (try to first run all the cells and then install missing libraries)

In [ ]:
import sys
# !{sys.executable} -m pip install torch torchvision
# !{sys.executable} -m pip install seaborn
# !{sys.executable} -m pip install imgaug
# !{sys.executable} -m pip install albumentations
# ...

In [ ]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = ''
 
%matplotlib inline
%load_ext autoreload
%autoreload 2
 
from os.path import join as pjoin
import glob
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from matplotlib import pyplot as plt
import matplotlib as mpl
from skimage import io
import seaborn as sns
import urllib.request
mpl.rcParams['figure.figsize'] = (15,8)
mpl.rcParams['image.cmap'] = 'inferno'
 
ROOT_DIR = os.getcwd()
DATA_DIR = pjoin(ROOT_DIR, 'data')

#### Download the dataframes:

In [ ]:
!mkdir '{DATA_DIR}'
!curl --header 'Host: storage.googleapis.com' --user-agent 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:63.0) Gecko/20100101 Firefox/63.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --header 'Upgrade-Insecure-Requests: 1' 'https://storage.googleapis.com/kaggle-datasets/24658/194179/train.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1543686099&Signature=hwSthMO3LEr7oP1ni8wyWGusb3Qdj1%2FS94fn3bflvtbzfVXKAcd%2FtR8XFiP8pIrDhx79SLpDYGh57NLM7sb3Ea0LCkh9kFlW%2BnBXSo4jOTPDLcVZbe%2B54cp3%2Faddb1PM2%2Fq7W4pzsIV8i8hMsq2XeqU%2F%2BQcPvaiUfzaUU6xkpPQ6VeaQrW9h9XR3rYbjoJECn5KfKuV0pOHpHA%2Fbg6oyt57cOPuQ3PRS7HyLBMxZKY27pcM6rXJuUGTObr60G4Ywk3kp2JYDVGj6kIUA4bu%2BB%2FxefEPF6pgkjQo7T3SwLXpGhlJ8jBvULiXjUi0N3UC6REzWxzCKeGkvxky4YP7keg%3D%3D' --output '{DATA_DIR}/train.csv.zip'
!curl --header 'Host: storage.googleapis.com' --user-agent 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:63.0) Gecko/20100101 Firefox/63.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --header 'Upgrade-Insecure-Requests: 1' 'https://storage.googleapis.com/kaggle-datasets/24658/194179/test.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1543686125&Signature=QI1fcdpPywTwQumWCIMsOz2JrVVx9ADvSf7JmnWcOLqGHVpnJKu%2FRasnoSHpuTb6jqcvnMM%2BZsl21%2FQLhIhp9fdE4%2BDBW2ZRTAnBDK1lz6NR6B4TJMQRAWogTNkJ9BkH6OeGOm%2FF%2F219%2BAWRKajkn3DL%2FBP5nynC%2B4RpBet4%2FRwGroMVuomjBpBE5eqC35f8nbRrnJNozVdBa8Ffy4RVm9Gu7fiK6zIKqIn%2Bc2LkXkhgcrT2nmwBBsriwcpv%2F%2FWBd4%2FeHV6sAE%2BIAcXEDOMQ4TQtYovXrhjDRe30SkjWlI8g79LQp%2FmOqAAe7yjjHb%2FO3XOiLExo3R2sPe8vwUUm%2Bw%3D%3D' --output '{DATA_DIR}/test.csv.zip'

#### ! Please unzip received files into the same directory

### EDA

Create utility functions:

In [ ]:
def default_image_file_loader(path):
#     return Image.open(path).convert('RGB')
#     return plt.imread(path)
    return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

def read_image(row, mode='url', datadir=DATA_DIR):
    if mode == 'url':
        return io.imread(row.url)
    else:
        return default_image_file_loader(pjoin(datadir, '{}.jpg'.format(row.id)))

def look_at_random(df, n=5, mode='url', datadir=DATA_DIR,
                   is_train=True, images_per_row=4, figsize=(20,10)):
    mpl.rcParams["figure.figsize"] = figsize
    datadir = pjoin(datadir, 'train') if is_train else pjoin(datadir, 'test')
    df_size = len(df['id'].unique())
    # make grid
    num_cols = images_per_row
    num_rows = (n - 1) // images_per_row + 1
    i = 1
    while i <= n:
        ind = np.random.randint(0, df_size)
        row = df.loc[ind]
        try:
            image = read_image(row, mode=mode, datadir=datadir)
            landmark = int(row.landmark_id) if is_train else '?'
            plt.subplot(num_rows, num_cols, i)
            plt.axis('off')
            plt.imshow(image)
            plt.title('{}\nclass: {}'.format(row.id, landmark))
            i += 1
        except:
            pass
    plt.show()

def get_images_ids_from_class(landmark_id, df):
    return list(df[df.landmark_id == landmark_id].id)
            
def look_at_randoms_from_class(df, landmark_id, n=5, mode='url', datadir=DATA_DIR,
                               images_per_row=4, figsize=(20,10)):
    mpl.rcParams["figure.figsize"] = figsize
    images_ids = get_images_ids_from_class(landmark_id, df)
    # make grid
    num_cols = images_per_row
    num_rows = (n - 1) // images_per_row + 1
    i = 1
    while i <= n:
        img_id = np.random.choice(images_ids)
        row = df[df.id == img_id].iloc[0]
        try:
            image = read_image(row, mode=mode, datadir=pjoin(datadir, 'train'))
            plt.subplot(num_rows, num_cols, i)
            plt.axis('off')
            plt.imshow(image)
            plt.title('{}\nclass: {}'.format(img_id, landmark_id))
            i += 1
        except:
            pass
    plt.show()
    
def get_most_frequent_landmarks(df, top=5, n=5, mode='url', datadir=DATA_DIR,
                                reverse=False, figsize=(20,10)):
    """List of the most/least frequent landmarks
    """
    if reverse:
        top_landmarks = pd.DataFrame(df.landmark_id.value_counts().tail(top))
    else:
        top_landmarks = pd.DataFrame(df.landmark_id.value_counts().head(top))
    
    top_landmarks.reset_index(inplace=True)
    top_landmarks.columns = ['landmark_id', 'num']
    plt.figure(figsize = figsize)
    plt.title('Most frequent landmarks' if not reverse else \
              'Least frequent landmarks')
    sns.set_color_codes("pastel")
    sns.barplot(x="landmark_id", y="num", data=top_landmarks,
                label="Count")
    plt.show()
    if n > 0:
        for _, row in top_landmarks.iterrows():
            print('\n\t{} images of class {}'.format(row.num, row.landmark_id))
            look_at_randoms_from_class(df, row.landmark_id,
                                       n=n, mode=mode, datadir=datadir,
                                       images_per_row=4,
                                       figsize=(20,10))
            
def get_random(df, n=5, mode='url', datadir=DATA_DIR, is_train=True):
    datadir = pjoin(datadir, 'train') if is_train else pjoin(datadir, 'test')
    df_size = len(df['id'].unique())
    images, landmarks = [], []
    i = 1
    while i <= n:
        ind = np.random.randint(0, df_size)
        row = df.loc[ind]
        try:
            image = read_image(row, mode=mode, datadir=datadir)
            landmark = int(row.landmark_id) if is_train else '?'
            images.append(image)
            landmarks.append(landmark)
            i += 1
        except:
            pass
    return images, landmarks

def get_random_from_class(df, landmark_id, n=5, mode='url', datadir=DATA_DIR):
    images = []
    images_ids = get_images_ids_from_class(landmark_id, df)
    i = 1
    while i <= n:
        img_id = np.random.choice(images_ids)
        row = df[df.id == img_id].iloc[0]
        try:
            image = read_image(row, mode=mode, datadir=pjoin(datadir, 'train'))
            images.append(image)
            i += 1
        except:
            pass
    return images, [landmark_id] * n

def visualize_images(images, landmarks=None, images_per_row=4, figsize=(20,10)):
    mpl.rcParams["figure.figsize"] = figsize
    # make grid
    n = len(images)
    num_cols = images_per_row
    num_rows = (n - 1) // images_per_row + 1
    i = 1
    for i, image in enumerate(images):
        plt.subplot(num_rows, num_cols, i+1)
        plt.axis('off')
        s = '\nclass {}'.format(landmarks[i]) if landmarks is not None else ''
        plt.title(str(image.shape[:2]) + s)
        plt.imshow(image, cmap='gray' if image.ndim == 2 else None)
    plt.show()

## Exploring

For the first stage we won't need all the images to be downloaded locally. We can analyse and preprocess data using only the dataframes. If you want to work with this dataset, it may be reasonable to first clean data and then download images that are relevant. 

In [ ]:
MODE = 'url' # may also be 'files' if you have image downloaded locally

train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

train_size = len(train_df.id.unique())
num_classes = len(train_df.landmark_id.unique())
test_size = len(test_df.id.unique())
print('{} train images from {} classes.'.format(train_size, num_classes))
print('{} test images.'.format(test_size))

In [ ]:
train_df.head()

In [ ]:
test_df.head()

To get familiar with images:

In [ ]:
look_at_random(train_df, n=5, mode=MODE, datadir=DATA_DIR, is_train=True,
               images_per_row=5, figsize=(20,100))

look_at_random(test_df, n=5, mode=MODE, datadir=DATA_DIR, is_train=False,
               images_per_row=5, figsize=(20,10))

In [ ]:
landmark_id = '0'
look_at_randoms_from_class(train_df, landmark_id, mode=MODE, datadir=DATA_DIR,
                           n=5, images_per_row=5, figsize=(20,8))

Let's check which landmarks are the most popular and how many examples are there, and also let's check the least popular classes: 

In [ ]:
get_most_frequent_landmarks(train_df, top=50, n=0, mode='url',
                            datadir=DATA_DIR, figsize=(20,8))
get_most_frequent_landmarks(train_df, top=50, n=0, reverse=True,
                            datadir=DATA_DIR, figsize=(20,8))

Some utility functions again...

In [ ]:
def p_quantile(x, p):
    from math import floor
    """
    0 <= p <= 1
    """
    x = sorted(x)
    n = len(x)
    
    m = n * p
    if m % 1 == 0:
        m = int(m)
        return (x[m-1] + x[m]) / 2
    else:
        return x[int(floor(m))]

def class_distr(train_df, bins, q=0.15):
    stats = train_df.landmark_id.value_counts(ascending=True)
    mean, lower_q, upper_q = stats.mean(), p_quantile(stats, q), p_quantile(stats, 1-q)
    print('mean: {} images per class, lower {}%-quantile: {}, upper {}%-quantile: {}'.\
          format(mean, q*100, lower_q, q*100, upper_q))
    values = []
    for (x, y) in zip(bins[:-1], bins[1:]):
        value = stats[stats <= y][stats > x].count()
        print('{:5} < n <= {:<5}: {}'.format(x, y, value))
        values.append(value)
    plt.step(bins[:-1], values)
    plt.vlines(mean, 0, max(values), 'gray', 'dashed')
    plt.vlines(lower_q, 0, max(values), 'k', 'dashed')
    plt.vlines(upper_q, 0, max(values), 'k', 'dashed')
    plt.xlabel('Images in class')
    plt.xlabel('Number of classes')
    plt.show()

def remove_outliers_landmarks(df, min_count=-np.inf, max_count=np.inf):
    counts = df.landmark_id.value_counts()
    landmarks_to_remove = list(counts[counts <= min_count].index) + \
                          list(counts[counts >= max_count].index)
    print('Removed {} landmarks &'.format(len(landmarks_to_remove)))
    print('        {} images'.format(len(df[df.landmark_id.isin(landmarks_to_remove)])))
    new_df = df[~df.landmark_id.isin(landmarks_to_remove)]
    print('Now we have {} images from {} classes.'.format(len(new_df.id.unique()),
                                                          len(new_df.landmark_id.unique())))
    return new_df

def shorten_for_demo(df, max_num_landmarks=1000, max_count=np.inf):
    print('\nRandom choice for shortening...')
    num_landmarks = len(df.landmark_id.unique())
    if num_landmarks < max_num_landmarks:
        landmarks_indices = np.arange(num_landmarks)
    else:
        landmarks_indices = np.random.choice(range(num_landmarks), max_num_landmarks, replace=False)
    total_ids_to_save = []
    for landmark_id, count in df.landmark_id.value_counts().iloc[landmarks_indices].iteritems():
        landmark_images_ids = df[df.landmark_id == landmark_id].id
        if count < max_count:
            ids_to_save = list(landmark_images_ids)
        else:
            random_ids = np.random.choice(landmark_images_ids, max_count, replace=False)
            ids_to_save = list(random_ids)
        total_ids_to_save.extend(ids_to_save)
    new_df = df[df.id.isin(total_ids_to_save)]
    print('Now we have {} images'.format(len(new_df)))
    print('            {} landmarks'.format(len(new_df.landmark_id.unique())))
    return new_df

### Obvious: remove landmarks with very few examples in class (say less then 10)

In [ ]:
th = 10
train_df_filtered = remove_outliers_landmarks(train_df, min_count=th)
train_size = len(train_df_filtered.id.unique())
num_classes = len(train_df_filtered.landmark_id.unique())

Look at the approximate distribution of the number of images in class:

In [ ]:
q = 0.1 # quantile to visualize for understanding
bins = [0, 22, 50, 100, 199, 501, 1000, 1995, 5011, 10000]
class_distr(train_df_filtered, bins, q)

You may have different goals regarding the dataset. But in many cases we need to make it balanced. So e.g. we can cut dataset such that only the classes of size between lower and upper 10% quantiles remain.

In [ ]:
stats = train_df_filtered.landmark_id.value_counts(ascending=True)
mean, lower_q, upper_q = stats.mean(), p_quantile(stats, q), p_quantile(stats, 1-q)
train_short_df = remove_outliers_landmarks(train_df_filtered,
                                           min_count=lower_q, max_count=upper_q)
train_short_df.head()

For the demonstration of image preprocessing we'll download 3 random classes:

In [ ]:
np.random.seed(0)
train_demo_df = shorten_for_demo(train_short_df, max_num_landmarks=3, max_count=upper_q)
train_demo_df.head()

In [ ]:
TRAIN_DATA_DIR = pjoin(DATA_DIR, 'train')
if not os.path.exists(TRAIN_DATA_DIR):
    os.makedirs(TRAIN_DATA_DIR)
    for i, row in train_demo_df.iterrows():
        try:
            filename, url, _ = row
            urllib.request.urlretrieve(url, pjoin(TRAIN_DATA_DIR, '{}.jpg'.format(filename)))
        except:
            pass
else:
    train_demo_df = pd.read_csv(pjoin(DATA_DIR, 'train_demo.csv'))

Sometimes the links become broken so we need to get rid of it. Because otherwise it may obstruct the process. We'll check whether there is a row in dataframe with no corresponding image in the local folder and remove it. 

In [ ]:
def remove_broken_links(df, img_dir):
    return df.drop(df.index[[i for i, idx in enumerate(df.id.values)
                    if not os.path.exists(pjoin(img_dir,'{}.jpg'.format(idx)))]])

print('Number of examples train table has')
print('\tbefore cleaning: {}'.format(len(train_demo_df.index)))

train_demo_df = remove_broken_links(train_demo_df, TRAIN_DATA_DIR)
train_demo_df = train_demo_df.reset_index().drop(columns='index')
train_demo_df.to_csv(pjoin(DATA_DIR, 'train_demo.csv'), index=False)

print('\tafter cleaning: {}'.format(len(train_demo_df.index)))

In [ ]:
train_demo_df.groupby(train_demo_df.landmark_id)[['id']].count()

In [ ]:
MODE = 'file'
landmarks = sorted(list(train_demo_df.landmark_id.unique()))

In [ ]:
for landmark_id in landmarks:
    look_at_randoms_from_class(train_demo_df, landmark_id, mode=MODE,
                               datadir=DATA_DIR,
                               n=5, images_per_row=5, figsize=(20,8))

So now let's imagine that this data is all we have. We want to make it as representative as possible.

---

#### Augmentations | imgaug

In [ ]:
import imgaug as ia
from imgaug import augmenters as iaa
import imgaug.augmenters.size as iaas

def get_augs():
    iaa_Blur = iaa.OneOf([
                    iaa.GaussianBlur((0, 1.5)), # blur images with a sigma between 0 and 1.5
                    iaa.AverageBlur(k=(2, 4)),  # blur image using local means with kernel sizes between 2 and 7
                    iaa.MedianBlur(k=(3, 5)),   # blur image using local medians with kernel sizes between 3 and 5
    ])
    iaa_Sharpen = iaa.OneOf([
                    iaa.Sharpen(alpha=(0, 0.5), lightness=(0.75, 1.5)), # sharpen images
                    iaa.Emboss(alpha=(0, 0.5), strength=(0, 0.5)),      # emboss images
    ])
    iaa_Noise = iaa.OneOf([
                    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.03*255), per_channel=0.1), # add gaussian noise to images
                    iaa.Dropout((0, 0.02), per_channel=0.1), # randomly remove up to 10% of the pixels
    ])
    iaa_Affine = iaa.OneOf([
                    iaa.Affine(
                        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},               # scale images to 80-120% of their size (per axis)
                        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate by -20 to +20 percent (per axis)
                        rotate=(-45, 45), # rotate by -45 to +45 degrees
                        shear=(-10, 10),  # shear by -10 to +10 degrees
                        order=[0, 1],     # use nearest neighbour or bilinear interpolation (fast)
                        cval=(0),
                        mode='constant'   # use zeros for padding
                    ),
#                     iaa.PerspectiveTransform(scale=(0.01, 0.1)) # this is a very slow transformation
    ])
    iaa_HSV = iaa.SomeOf((0,4),[
                    iaa.Grayscale(alpha=(0.0, 0.5)),  
                    iaa.Add((-10, 10), per_channel=0.01), # change brightness of images (by -10 to 10 of original value)
                    iaa.AddToHueAndSaturation((-10, 10)), # change hue and saturation
                    iaa.Sequential([
                       iaa.ChangeColorspace(from_colorspace="RGB",  # additioinally change saturation
                                            to_colorspace="HSV"),
                       iaa.WithChannels(1, iaa.Add((-100, 80))),
                       iaa.ChangeColorspace(from_colorspace="HSV",
                                            to_colorspace="RGB")
                    ]),
                    iaa.ContrastNormalization((0.7, 1.6), per_channel=0), # improve or worsen the contrast
                    iaa.Multiply((0.7, 1.2), per_channel=0.01),    # change value
                    iaa.FrequencyNoiseAlpha(                       # similar to previous ones
                        exponent=(-2, 2),
                        first=iaa.Multiply((0.85, 1.2), per_channel=False),
                        second=iaa.ContrastNormalization((0.5, 1.7))),
    ])
    iaa_Simple = iaa.SomeOf((0,2), [
                iaa.Fliplr(0.5),     # horizontally flip 50% of all images
                iaa.CropAndPad(      # crop
                    percent=(-0.05, 0.1),
                    pad_mode='constant',
                    pad_cval=(0)
                ),
    ])
        
    augs = {
        'simple': iaa_Simple,
        'affine': iaa_Affine,
        'hsv': iaa_HSV,
        'sharpen': iaa_Sharpen,
        'blur': iaa_Blur,
        'noise': iaa_Noise,
    }
    return augs

def get_final_transform():
    return iaa.Sequential([iaa.Crop(percent=(0,0.2), keep_size=False),
                           iaa.Scale({"height": 512, "width": "keep-aspect-ratio"})])

In [ ]:
mpl.rcParams["figure.figsize"] = (20, 20)
augs = get_augs()
final = get_final_transform()
augs_keys = list(augs.keys())
print('So we have: {}'.format(augs_keys))
landmark_id = landmarks[0]
imgs, _ = get_random_from_class(train_demo_df, landmark_id, mode=MODE,
                                datadir=DATA_DIR, n=len(augs_keys))
imgs = [np.array(img) for img in imgs]

In [ ]:
for i in range(len(augs_keys)):
    print(augs_keys[i])
    img = imgs[i]
    grid = iaa.Sequential([augs[augs_keys[i]], final]).draw_grid([img], cols=4, rows=2)
    plt.imshow(grid.astype(np.uint8))
    plt.show()

In [ ]:
img = np.array(get_random_from_class(train_demo_df, landmark_id, mode=MODE, datadir=DATA_DIR, n=1)[0][0])
total_augs = iaa.Sequential([augs['hsv'],
                             iaa.OneOf([augs['sharpen'], augs['blur'], augs['noise']]),
                             iaa.SomeOf((0,2), [augs['affine'], augs['simple']])])
grid = iaa.Sequential([total_augs, final]).draw_grid([img], cols=4, rows=4)
plt.imshow(grid.astype(np.uint8))
plt.show()

---

### Augmentations | torch & torchvision <3

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import functional as F

In [ ]:
class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        """
        Parameters
        ----------
        tensor : Tensor
            Tensor image of size (C, H, W) to be normalized.
            
        Returns
        -------
        Tensor
            Unnormalized image.
        """
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
            # The normalize code -> t.sub_(m).div_(s)
        return tensor

class SquareCenterCrop(object):
    """Crops the given PIL Image as a square at the center to save maximum.
    """

    def __init__(self):
        pass

    def __call__(self, img):
        """
        Parameters
        ----------
        img : PIL Image
            Image to be cropped.
            
        Returns
        -------
        PIL Image
            Cropped image.
        """
        h, w = img.size
        size = min(h, w)
        size = (int(size), int(size))
        return F.center_crop(img, size)

    def __repr__(self):
        return self.__class__.__name__ + '(size={0})'.format(self.size)
    
class LandmarksTransforms():
    CENTER_CROP_SHAPE = [1024, 1024]
    FINAL_SIZE = 224
    MEAN = [0.485, 0.456, 0.406]
    STD = [0.229, 0.224, 0.225]
    BRIGTHNESS = 0.5
    CONTRAST = 0.5
    SATURATION = 0.3
    HUE = 0.05
    SCALES = (0.08, 1.0)
    RATIOS = (0.8, 1.2)
    
    def __init__(self, mean=None, std=None):
        if mean is not None:
            self.MEAN = mean
        if mean is not None:
            self.STD = std
        self.augment = transforms.Compose([
                            SquareCenterCrop(),
                            transforms.RandomResizedCrop(self.FINAL_SIZE,
                                                         self.SCALES, self.RATIOS),
                            transforms.RandomHorizontalFlip(),
                            transforms.ColorJitter(self.BRIGTHNESS, self.CONTRAST,
                                                   self.SATURATION, self.HUE)
                        ])
#         self.augment = iaa.Sequential([total_augs, final])
        self.justResize = transforms.Compose([
            SquareCenterCrop(),
            transforms.Resize(self.FINAL_SIZE)
        ])
        self.toTensor = transforms.ToTensor()
        self.normalize = transforms.Normalize(self.MEAN, self.STD)
        self.unnormalize = UnNormalize(self.MEAN, self.STD)

In [ ]:
def default_image_file_loader(path):
    return Image.open(path).convert('RGB')

class LandmarksDataset(Dataset):
    
    def __init__(self, dataframe, images_dir,
                 is_train=True, augment=True, name=None):
        """
        Parameters
        ----------
        dataframe: pd.DataFrame or str
            csv table (or path to it) with following columns:
                id : str
                    id and also it's an image .jpg file name in images_dir
                url : str
                    link to image
                landmark_id : str
                    class [if is_train == True]
        images_dir: str
            path to directory containing images
        is_train: bool
            flag defining whether the classes are provided in table
        augment: bool
            flag defining whether to apply augmentation transformations on images
        name: str
            just the name of dataset
        """
        if isinstance(dataframe, pd.DataFrame):
            self.df = dataframe
        elif isinstance(dataframe, str):
            self.df = pd.read_csv(dataframe)
        self.is_train = is_train
        self.augment = augment
        self.name = name
        
        assert set(['id', 'url']).issubset(set(self.df.columns)), \
             'Please, provide DataFrame with `id` and `url`'
        self.images_ids = self.df.id.values
        self.images_paths = list(map(lambda id: os.path.join(images_dir, '{}.jpg'.format(id)),
                                     self.images_ids))
        self.images_urls = self.df.url.values
        self.num_samples = len(self.images_ids)
        print('Dataset {} loaded'.format("'"+self.name+"'" if self.name is not None else ''))
        print('\t{} images'.format(self.num_samples))
        
        if self.is_train:
            assert 'landmark_id' in self.df, 'Please, provide DataFrame with `landmark_id`'
            self.classes_ids = self.df.landmark_id.values
            self.num_classes = len(set(self.classes_ids))
            self.class_mapping = dict(zip(set(self.classes_ids), range(self.num_classes)))
            self.classes = [self.class_mapping[cl] for cl in self.classes_ids]
            print('\t{} classes'.format(self.num_classes))
        else:
            print('\tclasses are not provided')
            
        self.image_loader = default_image_file_loader
        self.transforms = LandmarksTransforms()

    
    def __getitem__(self, index):    
        image_id = self.images_ids[index]
        image = self.image_loader(self.images_paths[index])
        if self.is_train and self.augment:
            image = self.transforms.augment(image)
        else:
            image = self.transforms.justResize(image)
        image = self.transforms.toTensor(image)
        image = self.transforms.normalize(image)
        if self.is_train:
            class_id = self.classes[index]
            return image_id, image, class_id
        else:
            return image_id, image
    
    def __len__(self):
        return len(self.images_paths)
    
    def stats(self):
        shapes = [cv2.imread(img_path).shape for img_path in self.images_paths]
        return shapes

In [ ]:
dataset = LandmarksDataset(train_demo_df, TRAIN_DATA_DIR,
                           is_train=True, augment=True, name='landmarks')

BATCH_SIZE = 4
NUM_WORKERS = 16
loader = DataLoader(dataset, shuffle=True,
                    batch_size=BATCH_SIZE,
                    num_workers=NUM_WORKERS)

In [ ]:
for (_, inputs, targets) in loader:
    for i, (img, cls) in enumerate(zip(inputs, targets)):
        img = dataset.transforms.unnormalize(img).numpy().transpose(1,2,0)
        plt.subplot(1, 4, i+1)
        plt.axis('off')
        plt.title(cls.item())
        plt.imshow(img)
    plt.show()
    break